In [1]:
#  #colab setting 
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# os.chdir('/content/drive/My Drive/Competitions/医学影像报告异常检测/code')

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import torch
import transformers
from transformers import AdamW,get_linear_schedule_with_warmup
import pandas as pd


In [3]:
train_path = "../input/track1_round1_train_20210222.csv"
test_path = "../input/track1_round1_testA_20210222.csv"
train_df = pd.read_csv(train_path,sep=',',header=None)
test_df = pd.read_csv(test_path,sep=',',header=None)

train_df[1]=train_df[1].apply(lambda x:x[1:-1])
train_df[2]=train_df[2].apply(lambda x:x[1:])

test_df[1] = test_df[1].apply(lambda x:x[1:-1])


In [4]:
print(len(train_df))
train_df.drop_duplicates(subset=[1,2],inplace=True)
print(len(train_df))

10000
10000


In [5]:
train_df.columns = ['index','text','label']

train_df.head(20)

,index,text,label
0,0|,623 328 538 382 399 400 478 842 698 137 492 26...,2
1,1|,48 328 538 382 809 623 434 355 382 382 363 145...,
2,2|,623 656 293 851 636 842 698 493 338 266 369 69...,15
3,3|,48 328 380 259 439 107 380 265 172 470 290 693...,
4,4|,623 328 399 698 493 338 266 14 177 415 511 647...,16
5,5|,80 328 328 54 172 439 741 380 172 842 698 177 ...,15
6,6|,48 322 795 856 374 439 48 328 443 380 597 172 ...,4 11 15
7,7|,623 328 659 486 582 162 711 289 606 405 809 78...,
8,8|,852 328 471 585 117 458 399 607 693 380 522 62...,6 12 14 15
9,9|,229 172 200 737 437 547 651 693 623 328 355 65...,1 3


In [6]:
def get_dumm(s):
    re=[0]*17
    if s=='' or s==" ":
        return re
    else:
        tmp=[int(i) for i in s.split(' ') if i!='']
        for i in tmp:
            re[i]=1
    return re

train_df['label2'] = train_df.label.apply(lambda x:get_dumm(x))
train_df.head()

,index,text,label,label2
0,0|,623 328 538 382 399 400 478 842 698 137 492 26...,2,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1|,48 328 538 382 809 623 434 355 382 382 363 145...,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2|,623 656 293 851 636 842 698 493 338 266 369 69...,15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3|,48 328 380 259 439 107 380 265 172 470 290 693...,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4|,623 328 399 698 493 338 266 14 177 415 511 647...,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
for i in range(17):
    train_df['label_onehot_%d'%(i+1)] = train_df.label2.apply(lambda x:x[i])

In [8]:
train_df.head()

,index,text,label,label2,label_onehot_1,label_onehot_2,label_onehot_3,label_onehot_4,label_onehot_5,label_onehot_6,...,label_onehot_8,label_onehot_9,label_onehot_10,label_onehot_11,label_onehot_12,label_onehot_13,label_onehot_14,label_onehot_15,label_onehot_16,label_onehot_17
0,0|,623 328 538 382 399 400 478 842 698 137 492 26...,2,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1|,48 328 538 382 809 623 434 355 382 382 363 145...,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2|,623 656 293 851 636 842 698 493 338 266 369 69...,15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3|,48 328 380 259 439 107 380 265 172 470 290 693...,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4|,623 328 399 698 493 338 266 14 177 415 511 647...,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
test_df.columns = ['index','text']
test_df.head()

,index,text
0,0|,852 328 697 538 142 355 582 800 728 4 647 169 ...
1,1|,380 358 343 654 171 832 47 832 690 693 48 563 ...
2,2|,751 335 834 582 717 583 585 693 623 328 107 38...
3,3|,623 328 649 582 488 12 578 623 538 382 382 265...
4,4|,83 293 398 797 382 363 145 424 693 698 800 691...


In [10]:

def get_words_probs(df_a,df_b,label_name):
    df_b = df_b.copy()
    m_data = df_a.copy()
    m_data['text'] = m_data['text'].apply(lambda x:x.split())
    
    word_ctr = {}
    word_count = {}

    for idx,i in enumerate(range(len(m_data))):
        words = m_data.text.values[i]
        words = [i for i in set(words)]
        label = m_data[label_name].values[i]
        for w in words:
            word_count[w]=word_count.get(w,0)+1
            if label==1 :  #词频大于3
                word_ctr[w]=word_ctr.get(w,0)+1
                
    for k  in  word_ctr.keys():
        word_ctr[k] = word_ctr[k]/word_count[k]

    df_b['text'] =  df_b['text'].apply(lambda x:x.split())
    df_b['word_count_list'] = df_b.text.apply(lambda x:[word_count.get(i,0) for i in x ])
    df_b['word_ctr_list'] = df_b.text.apply(lambda x:[word_ctr.get(i,0) for i in x ])

    import numpy as np
    df_b['%s_word_ctr_list_mean_fea'%label_name] = df_b['word_ctr_list'].apply(lambda x:np.mean(x))
    df_b['%s_word_ctr_list_max_fea'%label_name] = df_b['word_ctr_list'].apply(lambda x:np.max(x))
    df_b['%s_word_ctr_list_min_fea'%label_name] = df_b['word_ctr_list'].apply(lambda x:np.min(x))
    df_b['%s_word_ctr_list_std_fea'%label_name] = df_b['word_ctr_list'].apply(lambda x:np.std(x))
    df_b['%s_word_ctr_list_sum_fea'%label_name] = df_b['word_ctr_list'].apply(lambda x:np.sum(x))
    df_b['text'] =  df_b['text'].apply(lambda x:" ".join(x))
    
    del df_b['word_count_list'],df_b['word_ctr_list']

    return df_b

for i in range(17):
    label_name = 'label_onehot_%d'%(i+1)
    test_df = get_words_probs(train_df,test_df,label_name)
    
test_df.head()

,index,text,label_onehot_1_word_ctr_list_mean_fea,label_onehot_1_word_ctr_list_max_fea,label_onehot_1_word_ctr_list_min_fea,label_onehot_1_word_ctr_list_std_fea,label_onehot_1_word_ctr_list_sum_fea,label_onehot_2_word_ctr_list_mean_fea,label_onehot_2_word_ctr_list_max_fea,label_onehot_2_word_ctr_list_min_fea,...,label_onehot_16_word_ctr_list_mean_fea,label_onehot_16_word_ctr_list_max_fea,label_onehot_16_word_ctr_list_min_fea,label_onehot_16_word_ctr_list_std_fea,label_onehot_16_word_ctr_list_sum_fea,label_onehot_17_word_ctr_list_mean_fea,label_onehot_17_word_ctr_list_max_fea,label_onehot_17_word_ctr_list_min_fea,label_onehot_17_word_ctr_list_std_fea,label_onehot_17_word_ctr_list_sum_fea
0,0|,852 328 697 538 142 355 582 800 728 4 647 169 ...,0.124919,0.393008,0.054608,0.079149,4.122316,0.127946,0.403226,0.058020,...,0.167549,0.285714,0.089904,0.042000,5.529130,0.077433,0.147368,0.038760,0.028777,2.555291
1,1|,380 358 343 654 171 832 47 832 690 693 48 563 ...,0.112956,0.197067,0.079536,0.021437,3.501624,0.108086,0.158730,0.064725,...,0.205120,0.303226,0.159265,0.034133,6.358730,0.087905,0.156806,0.041667,0.030928,2.725062
2,2|,751 335 834 582 717 583 585 693 623 328 107 38...,0.124938,0.224359,0.062696,0.039191,2.373818,0.131235,0.224359,0.078652,...,0.199417,0.566116,0.100313,0.092820,3.788926,0.093853,0.241986,0.029197,0.044474,1.783205
3,3|,623 328 649 582 488 12 578 623 538 382 382 265...,0.130126,0.423272,0.052124,0.051302,8.067832,0.131398,0.338954,0.046332,...,0.203535,0.492278,0.089904,0.069743,12.619198,0.075658,0.152263,0.015444,0.025709,4.690812
4,4|,83 293 398 797 382 363 145 424 693 698 800 691...,0.122920,0.230356,0.045000,0.041414,5.039707,0.137787,0.403226,0.071305,...,0.176610,0.331435,0.054931,0.057410,7.241030,0.078502,0.147368,0.017427,0.023886,3.218583


In [11]:
features = [i for i in test_df.columns if i not in ['index','text']]
print(len(features))
print(features)

85
['label_onehot_1_word_ctr_list_mean_fea', 'label_onehot_1_word_ctr_list_max_fea', 'label_onehot_1_word_ctr_list_min_fea', 'label_onehot_1_word_ctr_list_std_fea', 'label_onehot_1_word_ctr_list_sum_fea', 'label_onehot_2_word_ctr_list_mean_fea', 'label_onehot_2_word_ctr_list_max_fea', 'label_onehot_2_word_ctr_list_min_fea', 'label_onehot_2_word_ctr_list_std_fea', 'label_onehot_2_word_ctr_list_sum_fea', 'label_onehot_3_word_ctr_list_mean_fea', 'label_onehot_3_word_ctr_list_max_fea', 'label_onehot_3_word_ctr_list_min_fea', 'label_onehot_3_word_ctr_list_std_fea', 'label_onehot_3_word_ctr_list_sum_fea', 'label_onehot_4_word_ctr_list_mean_fea', 'label_onehot_4_word_ctr_list_max_fea', 'label_onehot_4_word_ctr_list_min_fea', 'label_onehot_4_word_ctr_list_std_fea', 'label_onehot_4_word_ctr_list_sum_fea', 'label_onehot_5_word_ctr_list_mean_fea', 'label_onehot_5_word_ctr_list_max_fea', 'label_onehot_5_word_ctr_list_min_fea', 'label_onehot_5_word_ctr_list_std_fea', 'label_onehot_5_word_ctr_list_s

In [12]:
from sklearn import model_selection
skf = model_selection.StratifiedKFold(n_splits=5, random_state=4590, shuffle=True)
for fold, (trn_, val_) in enumerate(skf.split(X=train_df, y=train_df.label.values)):
    train_df.loc[val_, 'kfold'] = fold
        

/home/root1/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [13]:
new_train_df = []

for fold in range(5):
    df_a = train_df[train_df.kfold!=fold]
    df_b = train_df[train_df.kfold==fold]
    for i in range(17):
        label_name = 'label_onehot_%d'%(i+1)
        df_b = get_words_probs(df_a,df_b,label_name)
    new_train_df.append(df_b)
    
train_df = pd.concat(new_train_df,axis=0)
train_df.head()

,index,text,label,label2,label_onehot_1,label_onehot_2,label_onehot_3,label_onehot_4,label_onehot_5,label_onehot_6,...,label_onehot_16_word_ctr_list_mean_fea,label_onehot_16_word_ctr_list_max_fea,label_onehot_16_word_ctr_list_min_fea,label_onehot_16_word_ctr_list_std_fea,label_onehot_16_word_ctr_list_sum_fea,label_onehot_17_word_ctr_list_mean_fea,label_onehot_17_word_ctr_list_max_fea,label_onehot_17_word_ctr_list_min_fea,label_onehot_17_word_ctr_list_std_fea,label_onehot_17_word_ctr_list_sum_fea
0,0|,623 328 538 382 399 400 478 842 698 137 492 26...,2,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,1,0,0,0,...,0.164440,0.316399,0.053208,0.061719,7.893109,0.069383,0.153386,0.009756,0.029972,3.330386
4,4|,623 328 399 698 493 338 266 14 177 415 511 647...,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0.189166,0.334732,0.069841,0.060036,4.161647,0.084684,0.129310,0.040923,0.023791,1.863055
5,5|,80 328 328 54 172 439 741 380 172 842 698 177 ...,15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0.195058,0.316399,0.064815,0.054437,7.022088,0.079598,0.135647,0.038522,0.024237,2.865540
7,7|,623 328 659 486 582 162 711 289 606 405 809 78...,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0.180168,0.398872,0.078740,0.061147,7.747219,0.082768,0.135647,0.029412,0.028315,3.559021
14,14|,623 328 697 661 809 48 46 355 661 414 852 328 ...,0 1 8 10,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...",1,1,0,0,0,0,...,0.187897,0.325301,0.088875,0.056875,7.891675,0.065703,0.147959,0.020346,0.026922,2.759523


In [14]:
from sklearn.utils import shuffle

In [15]:
def data_aug(df):
    
    import random
    df = df.copy()
    df[2]=df[2].apply(lambda x:x.split())
    new_df = df.copy()
    m =  shuffle(df)
    m.reset_index(drop=True,inplace=True)
    new_df[0]=100000
    new_df[1] = new_df[1]+m[1]
    new_df[2] = new_df[2]+m[2]
    new_df[2]=new_df[2].apply(lambda x:" ".join(list(set(x))))
    
    return new_df

## config

In [16]:
# 建立分词器

import transformers

class config:
    DEVICE = "cuda"
    MAX_LEN = 200
    TRAIN_BATCH_SIZE = 30
    VALID_BATCH_SIZE = 30
    MODEL_PATH = "model.bin"
    EPOCHS = 10
    BERT_PATH = "new_bert"
    TOKENIZER = transformers.BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)


save_path = config.BERT_PATH+'_weight_v2'

## SEED

In [17]:
import random 
import torch  
import numpy as np
import os
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed = 42
seed_everything(seed)

## DataSet

In [18]:


class BERTDataset:
    def __init__(self,num_features,text1,text2, target):
        self.text1 = text1
        self.text2 = text2
        self.target = target
        self.tokenizer = config.TOKENIZER
        self.max_length = config.MAX_LEN
        self.num_features = num_features

    def __len__(self):
        return len(self.text1)
    def get_dumm(self,s):
        # print("print s",s)
        # print("s split",s.split(' '))

        re=[0]*17
        if s=='' or s==" ":
            return re
        else:
            tmp=[int(i) for i in s.split(' ') if i!='']
            for i in tmp:
                re[i]=1
        return re


    def __getitem__(self, item):
        text1 = str(self.text1[item])
        text1 = " ".join(text1.split())
        text2 = None

        inputs = self.tokenizer.encode_plus(
            text1,
            text2,
            add_special_tokens=True,
            max_length=self.max_length,
        )
        input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        mask_padding_with_zero=True
        attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = self.max_length - len(input_ids)
        pad_on_left = False
        pad_token=0
        pad_token_segment_id=0

        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            attention_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + attention_mask
            token_type_ids = ([pad_token_segment_id] * padding_length) + token_type_ids
        else:
            input_ids = input_ids + ([pad_token] * padding_length)
            attention_mask = attention_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
            token_type_ids = token_type_ids + ([pad_token_segment_id] )* padding_length
                                               
        label = self.target[item]
        # print("label",label)
        #print(label,[i for i in label])
        label=self.get_dumm(label)
#         print(label)



        
        return {
            "ids": torch.tensor(input_ids, dtype=torch.long),
            "mask": torch.tensor(attention_mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": torch.tensor(label, dtype=torch.float),
            "num_features":torch.from_numpy(self.num_features[item]).float()
        }
    


# class BERTDataset:
#     def __init__(self,text1,text2, target):
#         self.text1 = text1
#         self.text2 = text2
#         self.target = target
#         self.tokenizer = config.TOKENIZER
#         self.max_length = config.MAX_LEN

#     def __len__(self):
#         return len(self.text1)

#     def get_dumm(self,s):
#         # print("print s",s)
#         # print("s split",s.split(' '))

#         re=[0]*17
#         if s=='' or s==" ":
#             return re
#         else:
#             tmp=[int(i) for i in s.split(' ') if i!='']
#             for i in tmp:
#                 re[i]=1
#         return re


#     def __getitem__(self, item):
#         text1 = str(self.text1[item])
#         text1 = " ".join(text1.split())
#         text2 = None
#         # text2 = str(self.text2[item])
#         # text2 = " ".join(text2.split())

#         # inputs = self.tokenizer.encode_plus(
#         #     text1,
#         #     text2,
#         #     # add_special_tokens=True,
#         #     max_length=self.max_len,
#         #     pad_to_max_length=True,
#         #     truncation=True
#         # )

#         # ids = inputs["input_ids"]
#         # mask = inputs["attention_mask"]
#         # token_type_ids = inputs["token_type_ids"]


#         inputs = self.tokenizer.encode_plus(
#             text1,
#             text2,
#             add_special_tokens=True,
#             max_length=self.max_length,
#             truncate_first_sequence=True  # We're truncating the first sequence in priority
#         )
#         input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]

#         # The mask has 1 for real tokens and 0 for padding tokens. Only real
#         # tokens are attended to.
#         mask_padding_with_zero=True
#         attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

#         # Zero-pad up to the sequence length.
#         padding_length = self.max_length - len(input_ids)
#         pad_on_left = False
#         pad_token=0
#         pad_token_segment_id=0

#         if pad_on_left:
#             input_ids = ([pad_token] * padding_length) + input_ids
#             attention_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + attention_mask
#             token_type_ids = ([pad_token_segment_id] * padding_length) + token_type_ids
#         else:
#             input_ids = input_ids + ([pad_token] * padding_length)
#             attention_mask = attention_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
#             token_type_ids = token_type_ids + ([pad_token_segment_id] )* padding_length
                                               
#         label = self.target[item]
#         # print("label",label)
#         #print(label,[i for i in label])
#         label=self.get_dumm(label)


#         return {
#             "ids": torch.tensor(input_ids, dtype=torch.long),
#             "mask": torch.tensor(attention_mask, dtype=torch.long),
#             "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
#             "targets": torch.tensor(label, dtype=torch.float),
#         }

In [19]:
from transformers import BertPreTrainedModel,BertModel
# from transformers.modeling_roberta import RobertaModel


## Model

In [20]:

import transformers
import torch.nn as nn
import math
import torch.nn.functional as F


import transformers
import torch.nn as nn

class CustomBert(transformers.BertPreTrainedModel):
    def __init__(self, config):
        config.output_hidden_states = True
        config.num_labels = 2

        super(CustomBert, self).__init__(config)
        self.num_labels = config.num_labels
        self.bert = transformers.BertModel(config)
        self.dropout = nn.Dropout(p=0.2)
        self.high_dropout = nn.Dropout(p=0.5)

        n_weights = config.num_hidden_layers + 1+13
        weights_init = torch.zeros(n_weights).float()
        weights_init.data[:-1] = -3
        self.layer_weights = torch.nn.Parameter(weights_init)

#         self.classifier = nn.Linear(config.hidden_size, self.config.num_labels)
        self.one_classifier = nn.Linear(1024, 17)
    
        self.line2 = nn.Linear(768+85,1024)


        self.dp1 = nn.Dropout(0.4)
        self.bn1 = nn.BatchNorm1d(17)
        self.bn2 = nn.BatchNorm1d(768+85)
        self.bn3 = nn.BatchNorm1d(1024)

        self.init_weights()

    def forward(self,
                input_ids=None,
                attention_mask=None,
                token_type_ids=None,
                position_ids=None,
                num_features=None
                ):
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids
                            )

        hidden_layers = outputs[2]
        last_layers = outputs[0]
#         print(len(hidden_layers))

        add_emb =[ torch.mean(layer,dim=1) for layer in hidden_layers] #14
        
        cls_outputs = torch.stack([self.dropout(layer[:, 0, :]) for layer in hidden_layers]+add_emb,
                                  dim=2)
#         print(cls_outputs.size())
#         print(self.layer_weights.size())
        cls_output = (torch.softmax(self.layer_weights, dim=0) * cls_outputs).sum(-1)
        
#         linear_out = F.relu(self.line1(num_features))
#         linear_out = self.dp1(linear_out)
#         linear_out = self.bn1(linear_out)
        

        cls_output =  torch.cat((cls_output,num_features),axis=1)
        cls_output = self.bn2(cls_output)
        
        cls_output = F.relu(self.line2(cls_output))
        cls_output = self.bn3(cls_output)
        

        
            # multisample dropout (wut): https://arxiv.org/abs/1905.09788
        logits2 = torch.mean(torch.stack([
            self.one_classifier(self.high_dropout(cls_output))
            for _ in range(5)
        ], dim=0), dim=0)
        

        return logits2




In [21]:

class FGM():
    """
    对抗训练方法类
    """
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=0.5, emb_name='word_embeddings'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0 and not torch.isnan(norm):
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='word_embeddings'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

## train units && eval units

In [22]:
import torch
import torch.nn as nn
from tqdm import tqdm





def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs.view(-1,17), targets.view(-1, 17))


def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()
    fgm = FGM(model)
    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d["targets"]
        num_feas = d['num_features']


        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)
        num_feas = num_feas.to(device, dtype=torch.float)


        optimizer.zero_grad()
        outputs = model(input_ids=ids, attention_mask=mask,token_type_ids=token_type_ids,num_features=num_feas)

        loss = loss_fn(outputs, targets)
        loss.backward()

        fgm.attack() # 在embedding上添加对抗扰动
        outputs = model(input_ids=ids, attention_mask=mask,token_type_ids=token_type_ids,num_features=num_feas)
        loss_adv = loss_fn(outputs, targets)
        # 反向传播，并在正常的grad基础上，累加对抗训练的梯度
        loss_adv.backward() 
        # 恢复embedding参数
        fgm.restore() 


        optimizer.step()
        scheduler.step()


def eval_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets = d["targets"]
            num_feas = d['num_features']


            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)
            num_feas = num_feas.to(device, dtype=torch.float)


            outputs = model(input_ids=ids, attention_mask=mask,token_type_ids=token_type_ids,num_features=num_feas)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [23]:

from sklearn.metrics import f1_score,accuracy_score,recall_score
from sklearn.metrics import roc_curve, precision_recall_curve


# def threshold_search(y_true, y_proba, plot=False):
#     precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
#     thresholds = np.append(thresholds, 1.001) 
#     F = 2 / (1/precision + 1/recall)
#     best_score = np.max(F)
#     best_th = thresholds[np.argmax(F)]
#     if plot:
#         plt.plot(thresholds, F, '-b')
#         plt.plot([best_th], [best_score], '*r')
#         plt.show()
#     search_result = {'threshold': best_th , 'f1': best_score}
#     return search_result 

In [24]:
test_df.head()

,index,text,label_onehot_1_word_ctr_list_mean_fea,label_onehot_1_word_ctr_list_max_fea,label_onehot_1_word_ctr_list_min_fea,label_onehot_1_word_ctr_list_std_fea,label_onehot_1_word_ctr_list_sum_fea,label_onehot_2_word_ctr_list_mean_fea,label_onehot_2_word_ctr_list_max_fea,label_onehot_2_word_ctr_list_min_fea,...,label_onehot_16_word_ctr_list_mean_fea,label_onehot_16_word_ctr_list_max_fea,label_onehot_16_word_ctr_list_min_fea,label_onehot_16_word_ctr_list_std_fea,label_onehot_16_word_ctr_list_sum_fea,label_onehot_17_word_ctr_list_mean_fea,label_onehot_17_word_ctr_list_max_fea,label_onehot_17_word_ctr_list_min_fea,label_onehot_17_word_ctr_list_std_fea,label_onehot_17_word_ctr_list_sum_fea
0,0|,852 328 697 538 142 355 582 800 728 4 647 169 ...,0.124919,0.393008,0.054608,0.079149,4.122316,0.127946,0.403226,0.058020,...,0.167549,0.285714,0.089904,0.042000,5.529130,0.077433,0.147368,0.038760,0.028777,2.555291
1,1|,380 358 343 654 171 832 47 832 690 693 48 563 ...,0.112956,0.197067,0.079536,0.021437,3.501624,0.108086,0.158730,0.064725,...,0.205120,0.303226,0.159265,0.034133,6.358730,0.087905,0.156806,0.041667,0.030928,2.725062
2,2|,751 335 834 582 717 583 585 693 623 328 107 38...,0.124938,0.224359,0.062696,0.039191,2.373818,0.131235,0.224359,0.078652,...,0.199417,0.566116,0.100313,0.092820,3.788926,0.093853,0.241986,0.029197,0.044474,1.783205
3,3|,623 328 649 582 488 12 578 623 538 382 382 265...,0.130126,0.423272,0.052124,0.051302,8.067832,0.131398,0.338954,0.046332,...,0.203535,0.492278,0.089904,0.069743,12.619198,0.075658,0.152263,0.015444,0.025709,4.690812
4,4|,83 293 398 797 382 363 145 424 693 698 800 691...,0.122920,0.230356,0.045000,0.041414,5.039707,0.137787,0.403226,0.071305,...,0.176610,0.331435,0.054931,0.057410,7.241030,0.078502,0.147368,0.017427,0.023886,3.218583


In [25]:

test_df['label']=''


test_num_features = test_df[features].values

test_dataset = BERTDataset(num_features=test_num_features,text1=test_df.text.values,text2=None,target=test_df.label.values)


test_data_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=config.VALID_BATCH_SIZE, num_workers=1
    )



In [26]:
print(test_dataset.__getitem__(0))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'ids': tensor([101, 118, 106, 136, 140, 239, 138, 125, 312, 192, 604, 294, 180, 304,
        311, 187, 959, 174, 105, 118, 106, 136, 125, 112, 140, 621, 291, 414,
        379, 192, 160, 253, 119, 161, 102,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0

In [27]:
from sklearn.model_selection import KFold,GroupKFold,StratifiedKFold
from sklearn.metrics import precision_score , recall_score	, roc_auc_score

from sklearn.model_selection import KFold,GroupKFold
# gkf =  GroupKFold(n_splits=5)
# groups_by_text_id_list = train['qid'].tolist()




oof_df_list = []

try:
    os.mkdir(save_path)
except:
    pass 

oof_train_list = []
pred = [] 

# score

def ss_score(targets,outputs):
    
    targets = torch.tensor(targets,dtype=torch.float)
    outputs = torch.tensor(outputs,dtype=torch.float)
    loss = nn.BCEWithLogitsLoss()(outputs.view(-1,17), targets.view(-1, 17))
    score = 1- loss/17
    return loss.cpu().detach().numpy(),score


for fold in range(5):
    

    print("开始第 %d 折训练"%(fold+1))
    

    
    model_path = "%s/bert_wwm_model_%d.bin"%(save_path,fold)
    config.MODEL_PATH = model_path
#     tr_df = train_df.iloc[trn_idx]
#     va_df = train_df.iloc[val_idx]

    tr_df = train_df[train_df.kfold!=fold]
    va_df = train_df[train_df.kfold==fold]
    

    

    
    tr_df_num_feas = tr_df[features].values
    va_df_num_feas = va_df[features].values



    train_dataset = BERTDataset(num_features=tr_df_num_feas,
        text1=tr_df.text.values,text2=None,target=tr_df.label.values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config.TRAIN_BATCH_SIZE, num_workers=4,shuffle=True
    )

    valid_dataset = BERTDataset(num_features=va_df_num_feas,
     text1= va_df.text.values,text2=None,target=va_df.label.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=config.VALID_BATCH_SIZE, num_workers=1
    )

    device = torch.device(config.DEVICE)
    model = CustomBert.from_pretrained(config.BERT_PATH,num_labels=1)
    model.to(device)

    param_optimizer = list(model.named_parameters())
    # pretrain model param
    param_pre = [(n, p) for n, p in param_optimizer if 'bert' in n]
    # dym param
    param_dym = [(n, p) for n, p in param_optimizer if 'layer_weights' in n ]
    # funsion layer param
    param_line = [p for n, p in param_optimizer if 'one_classifier' in n or 'line1' in n or 'line2' in n]
    # 不进行衰减的权重
    
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
         # pretrain model param
        # 衰减
        {'params': [p for n, p in param_pre if not any(nd in n for nd in no_decay)],
         'weight_decay': 0.01, 'lr': 2e-5
         },
        # 不衰减
        {'params': [p for n, p in param_pre if any(nd in n for nd in no_decay)],
         'weight_decay': 0.0, 'lr': 2e-5
         },
        # dym  layer
        # 衰减
        {'params': [p for n, p in param_dym if not any(nd in n for nd in no_decay)],
         'weight_decay': 0.01, 'lr': 1e-3
         },
        # 不衰减
        {'params': [p for n, p in param_dym if any(nd in n for nd in no_decay)],
         'weight_decay': 0.0, 'lr':1e-3
         },
 
        # linear  layer
        # 衰减
        {'params': [p for p in param_line ],
         'weight_decay': 0.0, 'lr': 1e-3
         }
    ]


    num_train_steps = int(len(tr_df) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    warmup_steps = int(num_train_steps*0.1)
    optimizer = AdamW(optimizer_parameters, lr=2e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,num_warmup_steps=warmup_steps,num_training_steps=num_train_steps
    )

    min_loss = 100000
    best_score =0
    
    for epoch in range(config.EPOCHS):
        train_fn(train_data_loader, model, optimizer, device, scheduler)
        outputs, targets = eval_fn(valid_data_loader, model, device)
#         loss= torch.nn.BCEWithLogitsLoss()(torch.tensor(outputs),torch.tensor(targets))
        loss,score = ss_score(targets,outputs)

        # result =  threshold_search(targets,outputs)
        # thr = result['threshold']
        # f1 = result['f1']
        # outputs = (outputs>thr).astype(int)
        # f1 = f1_score(targets,outputs)
        # precision = precision_score(targets, outputs)
        # recall =  recall_score(targets,outputs)

        print(f"loss = {loss}, score is {score}")


#         if loss<min_loss:
#             torch.save(model.state_dict(), config.MODEL_PATH)
#             min_loss = loss

        if score>best_score:
            best_score = score
            torch.save(model.state_dict(),config.MODEL_PATH)




    model.load_state_dict(torch.load( config.MODEL_PATH))
    outputs, targets = eval_fn(valid_data_loader, model, device)
    va_df['prob'] = outputs
    oof_train_list.append(va_df)

    outputs, targets = eval_fn(test_data_loader, model, device)
    pred.append(np.array(outputs))



开始第 1 折训练


Some weights of the model checkpoint at new_bert were not used when initializing CustomBert: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing CustomBert from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBert from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBert were not initialized from the model checkpoint at new_bert and are newly initialized: ['layer_weights', 'one_classifier.weight', 'one_classifier.bias', 'line2.wei

loss = 0.827002763748169, score is 0.9513527750968933


100%|██████████| 69/69 [00:04<00:00, 16.69it/s]


loss = 0.7685548067092896, score is 0.9547908902168274


...

100%|██████████| 66/66 [00:03<00:00, 16.57it/s]


loss = 0.7054612040519714, score is 0.9585022926330566


100%|██████████| 66/66 [00:03<00:00, 16.75it/s]


loss = 0.7066102027893066, score is 0.9584347009658813


100%|██████████| 100/100 [00:06<00:00, 16.63it/s]


In [28]:
oof_train = pd.concat(oof_train_list)
oof_train.head()

,index,text,label,label2,label_onehot_1,label_onehot_2,label_onehot_3,label_onehot_4,label_onehot_5,label_onehot_6,...,label_onehot_16_word_ctr_list_max_fea,label_onehot_16_word_ctr_list_min_fea,label_onehot_16_word_ctr_list_std_fea,label_onehot_16_word_ctr_list_sum_fea,label_onehot_17_word_ctr_list_mean_fea,label_onehot_17_word_ctr_list_max_fea,label_onehot_17_word_ctr_list_min_fea,label_onehot_17_word_ctr_list_std_fea,label_onehot_17_word_ctr_list_sum_fea,prob
0,0|,623 328 538 382 399 400 478 842 698 137 492 26...,2,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,1,0,0,0,...,0.316399,0.053208,0.061719,7.893109,0.069383,0.153386,0.009756,0.029972,3.330386,"[0.0028005423955619335, 0.0076747918501496315,..."
4,4|,623 328 399 698 493 338 266 14 177 415 511 647...,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0.334732,0.069841,0.060036,4.161647,0.084684,0.129310,0.040923,0.023791,1.863055,"[0.0012862442526966333, 0.007899697870016098, ..."
5,5|,80 328 328 54 172 439 741 380 172 842 698 177 ...,15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0.316399,0.064815,0.054437,7.022088,0.079598,0.135647,0.038522,0.024237,2.865540,"[0.004175003618001938, 0.004221336916089058, 0..."
7,7|,623 328 659 486 582 162 711 289 606 405 809 78...,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0.398872,0.078740,0.061147,7.747219,0.082768,0.135647,0.029412,0.028315,3.559021,"[0.00320462416857481, 0.0036855146754533052, 0..."
14,14|,623 328 697 661 809 48 46 355 661 414 852 328 ...,0 1 8 10,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...",1,1,0,0,0,0,...,0.325301,0.088875,0.056875,7.891675,0.065703,0.147959,0.020346,0.026922,2.759523,"[0.9999947547912598, 1.0, 0.009344044141471386..."


In [29]:
sub = np.average(pred, axis=0) 


In [30]:
m = sub.tolist()

In [33]:
def get_dumm(s):
    re=[0]*17
    if s=='' or s==" ":
        return re
    else:
        tmp=[int(i) for i in s.split(' ') if i!='']
        for i in tmp:
            re[i]=1
    return re


oof_train['label'] = oof_train['label'].apply(lambda x:get_dumm(x))

In [34]:
oof_train

,index,text,label,label2,label_onehot_1,label_onehot_2,label_onehot_3,label_onehot_4,label_onehot_5,label_onehot_6,...,label_onehot_16_word_ctr_list_max_fea,label_onehot_16_word_ctr_list_min_fea,label_onehot_16_word_ctr_list_std_fea,label_onehot_16_word_ctr_list_sum_fea,label_onehot_17_word_ctr_list_mean_fea,label_onehot_17_word_ctr_list_max_fea,label_onehot_17_word_ctr_list_min_fea,label_onehot_17_word_ctr_list_std_fea,label_onehot_17_word_ctr_list_sum_fea,prob
0,0|,623 328 538 382 399 400 478 842 698 137 492 26...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,1,0,0,0,...,0.316399,0.053208,0.061719,7.893109,0.069383,0.153386,0.009756,0.029972,3.330386,"[0.0028005423955619335, 0.0076747918501496315,..."
4,4|,623 328 399 698 493 338 266 14 177 415 511 647...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0.334732,0.069841,0.060036,4.161647,0.084684,0.129310,0.040923,0.023791,1.863055,"[0.0012862442526966333, 0.007899697870016098, ..."
5,5|,80 328 328 54 172 439 741 380 172 842 698 177 ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0.316399,0.064815,0.054437,7.022088,0.079598,0.135647,0.038522,0.024237,2.865540,"[0.004175003618001938, 0.004221336916089058, 0..."
7,7|,623 328 659 486 582 162 711 289 606 405 809 78...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0.398872,0.078740,0.061147,7.747219,0.082768,0.135647,0.029412,0.028315,3.559021,"[0.00320462416857481, 0.0036855146754533052, 0..."
14,14|,623 328 697 661 809 48 46 355 661 414 852 328 ...,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...","[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...",1,1,0,0,0,0,...,0.325301,0.088875,0.056875,7.891675,0.065703,0.147959,0.020346,0.026922,2.759523,"[0.9999947547912598, 1.0, 0.009344044141471386..."
21,21|,380 172 200 791 470 753 693 256 514 569 231 11...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0.817647,0.088235,0.108533,7.688602,0.083780,0.213592,0.027059,0.036084,3.267410,"[0.003285845974460244, 0.004417122341692448, 0..."
27,27|,852 328 501 355 360 265 478 162 498 289 169 40...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0.316399,0.089118,0.052943,10.974305,0.082929,0.153386,0.042620,0.022371,4.561104,"[0.0022288879845291376, 0.002418902702629566, ..."
31,31|,91 380 488 12 591 487 197 852 328 538 501 382 ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,...,0.305085,0.088875,0.052519,6.645423,0.086251,0.241199,0.043796,0.033514,3.105038,"[0.0037080259062349796, 0.003557736985385418, ..."
35,35|,380 315 809 623 328 160 380 419 789 439 852 81...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, ...",0,0,0,0,0,0,...,0.546296,0.099237,0.091041,11.624503,0.071593,0.171610,0.011194,0.028748,3.722838,"[0.0013001956976950169, 0.004078374244272709, ..."
41,41|,48 718 328 419 571 769 256 100 809 328 380 172...,"[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...",0,0,0,0,1,1,...,0.493888,0.053254,0.072358,13.445610,0.077398,0.159763,0.012225,0.026776,4.798670,"[0.00013342891179490834, 0.0001463307853555306..."


In [35]:
oof_train['score'] = oof_train.apply(lambda x:ss_score(x.label,x.prob)[1],axis=1)


In [36]:
oof_score = oof_train.score.mean()
print(oof_score)

0.95933291015625


In [37]:
test_df[2] = m
test_df.head()

,index,text,label_onehot_1_word_ctr_list_mean_fea,label_onehot_1_word_ctr_list_max_fea,label_onehot_1_word_ctr_list_min_fea,label_onehot_1_word_ctr_list_std_fea,label_onehot_1_word_ctr_list_sum_fea,label_onehot_2_word_ctr_list_mean_fea,label_onehot_2_word_ctr_list_max_fea,label_onehot_2_word_ctr_list_min_fea,...,label_onehot_16_word_ctr_list_min_fea,label_onehot_16_word_ctr_list_std_fea,label_onehot_16_word_ctr_list_sum_fea,label_onehot_17_word_ctr_list_mean_fea,label_onehot_17_word_ctr_list_max_fea,label_onehot_17_word_ctr_list_min_fea,label_onehot_17_word_ctr_list_std_fea,label_onehot_17_word_ctr_list_sum_fea,label,2
0,0|,852 328 697 538 142 355 582 800 728 4 647 169 ...,0.124919,0.393008,0.054608,0.079149,4.122316,0.127946,0.403226,0.058020,...,0.089904,0.042000,5.529130,0.077433,0.147368,0.038760,0.028777,2.555291,,"[0.00797245861031115, 0.013774707075208425, 0...."
1,1|,380 358 343 654 171 832 47 832 690 693 48 563 ...,0.112956,0.197067,0.079536,0.021437,3.501624,0.108086,0.158730,0.064725,...,0.159265,0.034133,6.358730,0.087905,0.156806,0.041667,0.030928,2.725062,,"[0.012245426326990128, 0.016009846050292254, 0..."
2,2|,751 335 834 582 717 583 585 693 623 328 107 38...,0.124938,0.224359,0.062696,0.039191,2.373818,0.131235,0.224359,0.078652,...,0.100313,0.092820,3.788926,0.093853,0.241986,0.029197,0.044474,1.783205,,"[0.0040807744022458795, 0.006838956056162715, ..."
3,3|,623 328 649 582 488 12 578 623 538 382 382 265...,0.130126,0.423272,0.052124,0.051302,8.067832,0.131398,0.338954,0.046332,...,0.089904,0.069743,12.619198,0.075658,0.152263,0.015444,0.025709,4.690812,,"[0.21206430047750474, 0.02738276617601514, 0.0..."
4,4|,83 293 398 797 382 363 145 424 693 698 800 691...,0.122920,0.230356,0.045000,0.041414,5.039707,0.137787,0.403226,0.071305,...,0.054931,0.057410,7.241030,0.078502,0.147368,0.017427,0.023886,3.218583,,"[0.03910873774439096, 0.029323529871180653, 0...."


In [40]:
test_df['index']=test_df['index'].apply(lambda x:x[:-1])
test_df.head()

,index,text,label_onehot_1_word_ctr_list_mean_fea,label_onehot_1_word_ctr_list_max_fea,label_onehot_1_word_ctr_list_min_fea,label_onehot_1_word_ctr_list_std_fea,label_onehot_1_word_ctr_list_sum_fea,label_onehot_2_word_ctr_list_mean_fea,label_onehot_2_word_ctr_list_max_fea,label_onehot_2_word_ctr_list_min_fea,...,label_onehot_16_word_ctr_list_min_fea,label_onehot_16_word_ctr_list_std_fea,label_onehot_16_word_ctr_list_sum_fea,label_onehot_17_word_ctr_list_mean_fea,label_onehot_17_word_ctr_list_max_fea,label_onehot_17_word_ctr_list_min_fea,label_onehot_17_word_ctr_list_std_fea,label_onehot_17_word_ctr_list_sum_fea,label,2
0,0,852 328 697 538 142 355 582 800 728 4 647 169 ...,0.124919,0.393008,0.054608,0.079149,4.122316,0.127946,0.403226,0.058020,...,0.089904,0.042000,5.529130,0.077433,0.147368,0.038760,0.028777,2.555291,,"[0.00797245861031115, 0.013774707075208425, 0...."
1,1,380 358 343 654 171 832 47 832 690 693 48 563 ...,0.112956,0.197067,0.079536,0.021437,3.501624,0.108086,0.158730,0.064725,...,0.159265,0.034133,6.358730,0.087905,0.156806,0.041667,0.030928,2.725062,,"[0.012245426326990128, 0.016009846050292254, 0..."
2,2,751 335 834 582 717 583 585 693 623 328 107 38...,0.124938,0.224359,0.062696,0.039191,2.373818,0.131235,0.224359,0.078652,...,0.100313,0.092820,3.788926,0.093853,0.241986,0.029197,0.044474,1.783205,,"[0.0040807744022458795, 0.006838956056162715, ..."
3,3,623 328 649 582 488 12 578 623 538 382 382 265...,0.130126,0.423272,0.052124,0.051302,8.067832,0.131398,0.338954,0.046332,...,0.089904,0.069743,12.619198,0.075658,0.152263,0.015444,0.025709,4.690812,,"[0.21206430047750474, 0.02738276617601514, 0.0..."
4,4,83 293 398 797 382 363 145 424 693 698 800 691...,0.122920,0.230356,0.045000,0.041414,5.039707,0.137787,0.403226,0.071305,...,0.054931,0.057410,7.241030,0.078502,0.147368,0.017427,0.023886,3.218583,,"[0.03910873774439096, 0.029323529871180653, 0...."


In [41]:
sub_id=test_df['index'].values
pres_all = sub.tolist()
str_w=''
with open(f'{save_path}/submit_{oof_score}.csv','w') as f:
    for i in range(len(sub_id)):
        pres_fold = pres_all[i]
        pres_fold=[str(p) for p in pres_fold]
        prob = " ".join(pres_fold)
        str_w+=sub_id[i]+'|'+','+'|'+prob+'\n'
    str_w=str_w.strip('\n')
    f.write(str_w)


In [42]:
test_df.to_csv(f"{save_path}/oof_test_{score}.csv")
oof_train.to_csv(f"{save_path}/oof_train_{score}.csv")